In [1]:
import xlutils
import databaker
import databaker.constants
from databaker.constants import *
import xypath
import databaker.databakersolo as ds
Dcelllookup = ds.overrides.Dcelllookup
batchcelllookup = ds.overrides.batchcelllookup


In [2]:
tableset = xypath.loader.table_set("gdpq3m2reftables_tcm77-425649.xls", extension='xls')
tabs = list(xypath.loader.get_sheets(tableset, "*"))
print("Table names", [tab.name  for tab in tabs])


Table names ['Content', 'Index', 'A1 AGGREGATES', 'A2 AGGREGATES', 'B1 CVM OUTPUT', 'B2 CVM OUTPUT', 'C1 EXPENDITURE', 'C2 EXPENDITURE', 'D INCOME', 'H1 TRADE', 'H2 TRADE', 'L GVAbp', 'M Alignment adjustments', 'N Financial Year Variables', 'O Selected imp def', 'P GDP per head']


In [29]:
conversionsegments = [ ]

for tab in tabs:
    if tab.name in ["A1 AGGREGATES"]:
        anchor = tab.excel_ref('A2').expand(DOWN).is_not_blank().is_not_whitespace().by_index(1)
        obs = anchor.fill(RIGHT).expand(DOWN).is_not_blank().is_not_whitespace()
        obs = obs - tab.excel_ref('B2').expand(DOWN).is_not_blank().is_not_whitespace().expand(RIGHT).expand(LEFT).expand(DOWN)        
        
        cdid_horizontal = anchor.shift(UP).fill(RIGHT).is_not_blank().is_not_whitespace()       
        cdid_vertical = anchor.expand(DOWN).is_blank()
        cdid = cdid_horizontal.waffle(cdid_vertical).is_not_blank().is_not_whitespace() | cdid_horizontal
        obs = obs - cdid
        obs = obs - obs.filter(contains_string('-'))

        dimensions = [ 
            (anchor.expand(DOWN).is_not_blank().is_not_whitespace().is_not_bold(), TIME, DIRECTLY, LEFT), 
            (anchor.expand(DOWN).is_not_blank().is_not_whitespace().is_bold(), 'Output', CLOSEST, ABOVE), 
            (tab.excel_ref('A3').expand(RIGHT).is_not_blank().is_not_whitespace(), "Cat1", DIRECTLY, ABOVE), 
            (tab.excel_ref('A2').expand(RIGHT).is_not_blank().is_not_whitespace().parent(), "Cat2", CLOSEST, LEFTRIGHT), 
            (cdid, 'CDID', DIRECTLY, ABOVE)
        ]
        
        #OBS <--- list of cells  (blue)
        #TIME <--- list of cells    (yellow)
        #Output <--- list of cells  (green)
        #Cat1 <-- list of cells (light blue)
        #Cat2 <-- list of cells (blue)
        #CDID <-- list of cells        
        conversionsegments.append((tab, dimensions, obs))

# this is the preview system
conversionsegment = conversionsegments[-1]
from databaker.jupybakeutils import savehtml
savehtml(conversionsegment, batchcelllookup, "preview.html")
        

opening file preview.html
table written
javascript calculated


In [5]:

conversionsegments = [ ]

for tab in tabs:
    if tab.name in ["A1 AGGREGATES", "A2 AGGREGATES"]:
        anchor = tab.excel_ref('A2').expand(DOWN).is_not_blank().is_not_whitespace().by_index(1)
        obs = anchor.fill(RIGHT).expand(DOWN).is_not_blank().is_not_whitespace()

        cdid_horizontal = anchor.shift(UP).fill(RIGHT).is_not_blank().is_not_whitespace()       
        cdid_vertical = anchor.expand(DOWN).is_blank()
        cdid = cdid_horizontal.waffle(cdid_vertical).is_not_blank().is_not_whitespace() | cdid_horizontal        
        
        dimensions = [ 
            (tab.excel_ref('A2').expand(RIGHT).is_not_blank().is_not_whitespace().parent(), "Expenditure", CLOSEST, LEFTRIGHT), 
            (tab.excel_ref('A3').expand(RIGHT).is_not_blank().is_not_whitespace(), "Exp1", DIRECTLY, ABOVE), 
            # constant type (None, MEASURETYPE, 'Number') # 

            (anchor.expand(DOWN).is_not_blank().is_not_whitespace().is_not_bold(), TIME, DIRECTLY, LEFT), 
            (anchor.expand(DOWN).is_not_blank().is_not_whitespace().is_bold(), 'Output', CLOSEST, ABOVE), 
            (cdid, 'CDID', DIRECTLY, ABOVE)
        ]
#        obs = obs - tab.excel_ref('B2').expand(DOWN).is_not_blank().is_not_whitespace().expand(RIGHT).expand(LEFT).expand(DOWN)        
#        obs = obs - cdid
        
#        obs = obs.is_not_blank().is_not_whitespace()

        conversionsegments.append((tab, [], obs))
        
#bake --preview recipe.py gdpq3m2reftables_tcm77-425649.xls "A1 AGGREGATES" None Y
#bake --preview recipe.py gdpq3m2reftables_tcm77-425649.xls "A1 AGGREGATES" None Q
        
conversionsegment = conversionsegments[-1]

from databaker.jupybakeutils import savehtml
savehtml(conversionsegment, batchcelllookup, "preview.html")

        
        

opening file preview.html
table written
javascript calculated


In [ ]:


# preview the cells in this batch
from databaker.jupybakeutils import dsubsets, tabletohtml, sidewindowhtmldisplay, inlinehtmldisplay, incrementdividNUM, inlinehtmljsactive

conversionsegment = conversionsegments[-1]

tab, dimensions, segment = conversionsegment
htmtable = tabletohtml(tab, dsubsets(dimensions, segment))
incrementdividNUM()
inlinehtmldisplay(htmtable, hide=False)
inlinehtmljsactive(conversionsegment, batchcelllookup)




In [30]:

# this currently does the lookups!
allvalues = [ ]

def procbatch(dimvalue, dimlabel):
    return [ c.value if c is not None else "" for c in dimvalue ]  # "2010²"

for tab, dimensions, segment in conversionsegments:
    obslist = list(segment.unordered_cells)  # list(segment) otherwise gives bags of one element
    dimvalues = [ procbatch(batchcelllookup(tab, obslist, dimension), dimension[1])  for dimension in dimensions ]
    dtuples = zip(*([procbatch(obslist, OBS)]+dimvalues))
    keys = [OBS] + [ dimension[1]  for dimension in dimensions ]  # the labels
    dvalues = [ dict(zip(keys, dtup))  for dtup in dtuples  if not isinstance(dtup[0], str) ]
    allvalues.extend(dvalues)
    
# In theory we can now call the template export to big CSV, like before at this point
# But now we should seek to plot the stats ourselves as a sanity check that the data is good
        

len(allvalues), allvalues[-1]
#conversionsegments[9][1]


(2024,
 {'Cat2': 'Value indices at current prices',
  'Output': 'Percentage change, latest quarter on corresponding quarter of previous year',
  'Cat1': 'Gross domestic product at market prices',
  'CDID': 'IHYO',
  -9: 1.5,
  -2: '2008 Q3'})

In [34]:
allvalues.sort(key=lambda X: str(X[-2]))
allvalues[:10]

[{'Cat2': 'Value indices at current prices',
  'Output': 'Seasonally adjusted',
  'Cat1': 'Gross value added at basic prices',
  'CDID': 'YBEX',
  -9: 52.8,
  -2: '1997 Q1'},
 {'Cat2': 'Implied deflators 2',
  'Output': 'Seasonally adjusted',
  'Cat1': 'Gross domestic expenditure',
  'CDID': 'YBFV',
  -9: 71.2,
  -2: '1997 Q1'},
 {'Cat2': 'Chained  volume indices',
  'Output': 'Seasonally adjusted',
  'Cat1': 'Market sector gross value added',
  'CDID': 'L48H',
  -9: 74.2,
  -2: '1997 Q1'},
 {'Cat2': 'Implied deflators 2',
  'Output': 'Seasonally adjusted',
  'Cat1': 'Gross domestic product at market prices',
  'CDID': 'YBGB',
  -9: 71.0,
  -2: '1997 Q1'},
 {'Cat2': 'Value indices at current prices',
  'Output': 'Seasonally adjusted',
  'Cat1': 'Gross domestic product at market prices',
  'CDID': 'YBEU',
  -9: 52.2,
  -2: '1997 Q1'},
 {'Cat2': 'Chained  volume indices',
  'Output': 'Seasonally adjusted',
  'Cat1': 'Gross value added at basic prices',
  'CDID': 'CGCE',
  -9: 73.8,
  -2:

In [ ]:


# alternatively, using pivot table (made by Peter)
import pandas as pd
df = ( 
    pd.DataFrame.from_records(mallvalues)
    .rename(columns={-6:"MEASURETYPE", -2:"TIME", -9:"OBS"})
    .assign(TIME=lambda df:df.TIME.astype(int))
    #.assign(TIME2=lambda df:df.TIME.astype(int)*2)
)

pdf = df.pivot_table("OBS", ["TIME"], ["Mergers", "MEASURETYPE", "Area"])
# ["Area Analysis of Acquisitions Abroad by UK Companies"].xs("Number", level="MEASURETYPE", axis=1).plot()
acqtypes = pdf.columns.get_level_values(0).unique()
valtypes = pdf.columns.get_level_values(1).unique()

acqtype = acqtypes[0]
valtype = valtypes[0]
pdf[acqtype][valtype].plot(legend=False, title=acqtype)
